# Imports

In [ ]:
# Move to the root
import os
cwd = os.getcwd()
if os.path.basename(cwd) != "cv-in-farming":
    os.chdir("../")
print(os.getcwd())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from src.image_processing import apply_template_matching
from src.dataloader import FurrowDataset
from utils.helpers import create_template, shift_pixels, generate_lane_pixels, overlay_coord, show_image

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Definitions

In [ ]:
class Intrinsics:
    def __init__(self, fx, fy, ppx, ppy, distortion, coeff):
        self.fx = fx
        self.fy = fy
        self.ppx = ppx
        self.ppy = ppy
        self.distortion = distortion
        self.coeff = coeff

intrinsics = Intrinsics(fx=618.75439453125, 
                        fy=616.88916015625,
                        ppx=324.08642578125,
                        ppy=241.37208557128906,
                        distortion="inverse_brown_conrady",
                        coeff=[0.0, 0.0, 0.0, 0.0, 0.0])

# Read Frames

In [ ]:
normalize = True
input_format = 'rgb'

dataset_args = {
    "data_path": "dataset/val/20210309_124809",
    "crop_down": False,
    "normalize": normalize,
    "input_format": input_format,
    "load_edge": False,
    "edge_width": 3,
    "load_time": False,
    "max_frames": 1000
}

folder = FurrowDataset(dataset_args)
print(folder)

# Get Edge Pixels

In [ ]:
i = 500

## Compute Pixel Coordinates

In [ ]:
item = folder.get_frame_files(i,
                              load_darr=True,
                              load_rgb=True,
                              load_drgb=False,
                              load_edge=False,
                              load_time=False)
rgb_img = np.array(item['rgb_img'])
depth_arr = item['depth_arr']
template = create_template(size=30, position=1)
pixels, corners, outliers = apply_template_matching(depth_arr.astype(np.float32), 
                                                    template,
                                                    y_step=25,
                                                    n_contours=200,
                                                    fit_type="curve",
                                                    verbose=1)

plt.figure(figsize=(10,10))
plt.imshow(rgb_img)
plt.scatter(corners[:,1], corners[:,0], color="black", marker="o")
plt.scatter(outliers[:,1], outliers[:,0], color="purple", marker="^")
plt.show()

## Load Precomputed Pixel Coordinates

In [ ]:
item = folder.get_frame_files(i,
                              load_darr=True,
                              load_rgb=True,
                              load_drgb=False,
                              load_edge=True,
                              load_time=False)
rgb_img = np.array(item["rgb_img"])
depth_arr = item["depth_arr"]
pixels = item["edge_pixels"]

In [ ]:
print(item["frame_id"])

# Shift Detection

In [ ]:
# Take a copy of pixel coordinate of corners shifted with shift3D vector
shifted2D = shift_pixels(depth_arr, pixels, intrinsics, shift3D=(-0.53,0,0))

# Generate lanes
lanes = generate_lane_pixels(shifted2D, pixels, num_lane=15)

In [ ]:
# Visualize in 2D
overlaid = overlay_coord(rgb_img, shifted2D, thickness=2, color='springgreen')
for lane_coords in lanes:
    overlaid = overlay_coord(overlaid, lane_coords, thickness=2, color='springgreen')
overlaid = overlay_coord(overlaid, pixels, thickness=2, color='springgreen')
show_image(overlaid, cmap='gray', ticks=False)

# Extra: Projection/Backprojection

In [ ]:
from utils.helpers import project, backproject

x = np.arange(640)
y = np.arange(480)
x, y = np.meshgrid(x, y)
coords = np.c_[y.ravel(), x.ravel()]
P = backproject(depth_arr, coords, intrinsics)
coords2 = project(P, intrinsics)
print(coords)
print(coords2)